<a href="https://colab.research.google.com/github/ansul1214/Open_Project_Winter_2025/blob/main/W1_Quantum_Tomography_23113029.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Environment Setup

In [1]:
!python -m pip install  pennylane numpy scipy pandas plotly tqdm nbformat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 70.7 MB/s eta 0:00:00


In [2]:
!python -m pip install jax~=0.6.0 jaxlib~=0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.7.2
    Uninstalling jaxlib-0.7.2:
      Successfully uninstalled jaxlib-0.7.2
  Attempting uninstall: jax
    Found existing installation: jax 0.7.2
    Uninstalling jax-0.7.2:
      Successfully uninstalled jax-0.7.2


In [3]:
import pennylane as qml
import numpy as np
import json
from typing import Dict, Any

# Task 2: Measurement Theory Pointer

To-Do: Prepare at minimum the computational basis (|0⟩, |1⟩), the Hadamard basis (|+⟩, |−⟩), and one phase-offset state (e.g., ( |0⟩ + i |1⟩ ) / √2). Document how you synthesize each state in circuit form and store a textual or JSON summary of the gates used. You may optionally include mixed states by applying depolarizing or amplitude damping channels.

|0> : [1 0].T - 0 bit Dirac notation


|1> : [0 1].T - 1 bit Dirac notation

Hadmard Basis:

These are the linear combinations of the standard basis |0> and |1>.

|+> = 1/√2(|0>+|1>)

|-> = 1/√2(|0>-|1>)

The |1> can be represented as a circuit of the not operator on the zero bit.

|1>=X|0>

X=array([[1, 0],
       [0, 1]])

For |+> = H|0>

 |-> = HS|0>

 For |0+i1>/√2
   
   |0+i1>/√2 = HS|0>


In [4]:
dev = qml.device("default.qubit", wires=1)

In [5]:
def prepare_state(state):
    if state == "|0>":
        pass
    elif state == "|1>":
        qml.PauliX(0)
    elif state == "|+>":
        qml.Hadamard(0)
    elif state == "|->":
        qml.PauliX(0)
        qml.Hadamard(0)
    elif state == "|0+i1>/√2":
        qml.Hadamard(0)
        qml.S(0)
    else:
        raise ValueError("Invalid state")

In [6]:
state_preparation_summary = {
    "|0>": [],
    "|1>": ["PauliX"],
    "|+>": ["Hadamard"],
    "|->": ["PauliX", "Hadamard"],
    "|0+i1>/√2": ["Hadamard", "S"]
}

with open("state_preparation.json", "w") as f:
    json.dump(state_preparation_summary, f, indent=2)


In [7]:
@qml.qnode(dev)
def noisy_plus_state(p=0.1):
    qml.Hadamard(0)
    qml.DepolarizingChannel(p, wires=0)
    return qml.density_matrix(wires=0)

In [8]:
import pathlib

def build_measurement_model(config_path: pathlib.Path) -> Dict[str, Any]:
    """
    Stub for constructing or loading the measurement operators you plan to use.
    Populate the return value with operator definitions, normalization checks, and metadata.
    """
    X=np.array([[0,1],[1,0]],dtype=complex)
    Y=np.array([[0,-1j],[1j,0]],dtype=complex)
    Z=np.array([[1,0],[0,-1]],dtype=complex)
    I=np.array([[1,0],[0,1]],dtype=complex)
    P0=np.array([[1,0],[0,0]],dtype=complex)
    P1=np.array([[0,0],[0,1]],dtype=complex)
    completeness=np.allclose(P0+P1,I)
    return {
        "type": "pauli_projective",
        "operators":{"X":X,"Y":Y,"Z":Z,"I":I},
        "Projections":{"P0":P0,"P1":P1},
        "completeness":completeness,
        "metadata": {
            "framework": "PennyLane",
            "description": "Single-qubit Pauli projective measurement model"
        }
    }


    # TODO: implement SIC POVM or Pauli projective operator assembly here.
    raise NotImplementedError("Create your measurement operator assembly here.")

# Helper Histogram

In [9]:
#@title helper functions for density matrix visualization

import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES = (
    (0, 1, 2), (0, 2, 3),  # bottom
    (4, 5, 6), (4, 6, 7),  # top
    (0, 1, 5), (0, 5, 4),
    (1, 2, 6), (1, 6, 5),
    (2, 3, 7), (2, 7, 6),
    (3, 0, 4), (3, 4, 7)
 )

def _phase_to_pi_string(angle_rad: float) -> str:
    """Format a phase angle as a simplified multiple of π."""
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple = angle_rad / np.pi
    frac = Fraction(multiple).limit_denominator(16)
    numerator = frac.numerator
    denominator = frac.denominator
    sign = "-" if numerator < 0 else ""
    numerator = abs(numerator)
    if denominator == 1:
        magnitude = f"{numerator}" if numerator != 1 else ""
    else:
        magnitude = f"{numerator}/{denominator}"
    return f"{sign}{magnitude}π" if magnitude else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density matrix (|ρ_ij| as bar height, phase as color)"):
    """Render a density matrix as a grid of solid histogram bars with phase coloring."""
    rho = np.asarray(rho)
    if rho.ndim != 2 or rho.shape[0] != rho.shape[1]:
        raise ValueError("rho must be a square matrix")

    dim = rho.shape[0]
    mags = np.abs(rho)
    phases = np.angle(rho)
    x_vals = np.arange(dim)
    y_vals = np.arange(dim)

    if basis_labels is None:
        basis_labels = [str(i) for i in range(dim)]

    meshes = []
    colorbar_added = False
    for i in range(dim):
        for j in range(dim):
            height = mags[i, j]
            phase = phases[i, j]
            x0, x1 = i - 0.45, i + 0.45
            y0, y1 = j - 0.45, j + 0.45
            vertices = (
                (x0, y0, 0.0), (x1, y0, 0.0), (x1, y1, 0.0), (x0, y1, 0.0),
                (x0, y0, height), (x1, y0, height), (x1, y1, height), (x0, y1, height)
            )
            x_coords, y_coords, z_coords = zip(*vertices)
            i_idx, j_idx, k_idx = zip(*_CUBE_FACES)
            phase_pi = _phase_to_pi_string(phase)
            mesh = go.Mesh3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                i=i_idx,
                j=j_idx,
                k=k_idx,
                intensity=[phase] * len(vertices),
                colorscale="HSV",
                cmin=-np.pi,
                cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(
                    title="phase ",
                    tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                    ticktext=["-π", "-π/2", "0", "π/2", "π"]
                ) if not colorbar_added else None,
                opacity=1.0,
                flatshading=False,
                hovertemplate=
                    f"i={i}, j={j}<br>|ρ_ij|={height:.3f}<br>arg(ρ_ij)={phase_pi}<extra></extra>",
                lighting=dict(ambient=0.6, diffuse=0.7)
            )
            meshes.append(mesh)
            colorbar_added = True

    fig = go.Figure(data=meshes)
    fig.update_layout(
        scene=dict(
            xaxis=dict(
                title="i",
                tickmode="array",
                tickvals=x_vals,
                ticktext=basis_labels
            ),
            yaxis=dict(
                title="j",
                tickmode="array",
                tickvals=y_vals,
                ticktext=basis_labels
            ),
            zaxis=dict(title="|ρ_ij|"),
            aspectratio=dict(x=1, y=1, z=0.7)
        ),
        title=title,
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.show()

# Task 3: QST Data Generation

In [10]:
STATE_ID_MAP = {
    "|0>": "zero",
    "|1>": "one",
    "|+>": "plus",
    "|->": "minus",
    "|0+i1>/√2": "phase"
}

In [11]:
from dataclasses import dataclass
from typing import List
import pathlib

In [12]:
@dataclass
class DatasetVariant:
    name: str
    circuit_summary: str
    measurement_model: str
    measurement_data_path: pathlib.Path
    metadata_path: pathlib.Path
    density_matrix_path: pathlib.Path

In [13]:
@dataclass
class DatasetVariant:
    name: str
    circuit_summary: str
    measurement_model: str
    measurement_data_path: pathlib.Path
    metadata_path: pathlib.Path
    density_matrix_path: pathlib.Path

In [14]:
SHOTS = 2000
NUM_QUBITS = 1   # change to >1 later

dev_counts = qml.device(
    "default.qubit",
    wires=NUM_QUBITS,
)

dev_dm = qml.device(
    "default.qubit",
    wires=NUM_QUBITS
)


In [15]:
@qml.set_shots(SHOTS)
@qml.qnode(dev_counts)
def measure_pauli_string(state_label: str, pauli_string: str):
    prepare_state(state_label)

    for wire, p in enumerate(pauli_string):
        if p == "X":
            qml.Hadamard(wire)
        elif p == "Y":
            qml.PhaseShift(np.pi / 2, wire)
            qml.Hadamard(wire)
        elif p == "Z":
            pass
        elif p == "I":
            pass

    return qml.counts(wires=range(NUM_QUBITS))


In [16]:
@qml.qnode(dev_dm)
def get_true_density_matrix(state_label: str):
    prepare_state(state_label)
    return qml.density_matrix(wires=range(NUM_QUBITS))

In [17]:
@qml.qnode(dev_dm)
def get_true_density_matrix(state_label: str):
    prepare_state(state_label)
    return qml.density_matrix(wires=range(NUM_QUBITS))

In [18]:
def normalize_counts(counts: dict):
    total = sum(counts.values())
    return {k: v / total for k, v in counts.items()}


In [19]:
def generate_measurement_dataset(variants: List[DatasetVariant]) -> None:
    """
    Populate each variant with measurement outcomes, metadata,
    and ground-truth density matrices.
    Designed to scale from single-qubit to multi-qubit systems.
    """

    # Measurement settings (extendable)
    pauli_settings = ["X", "Y", "Z"] if NUM_QUBITS == 1 else [
        "XX", "YY", "ZZ", "XY", "XZ", "YZ"
    ]


    for variant in variants:
        print("Processing state:",variant.name)
        print("Saving to:",variant.measurement_data_path)
        state = variant.name
        measurement_results = {}

        for setting in pauli_settings:
            counts = measure_pauli_string(state, setting)
            measurement_results[setting] = {
                "counts": counts,
                "probabilities": normalize_counts(counts)
            }

        # Ensure directories exist
        variant.measurement_data_path.parent.mkdir(parents=True, exist_ok=True)

        # Save measurement data
        np.save(
            variant.measurement_data_path,
            measurement_results,
            allow_pickle=True
        )

        # Save ground truth density matrix
        rho_true = get_true_density_matrix(state)
        np.save(variant.density_matrix_path, rho_true)

        # Save metadata
        metadata = {
            "state": state,
            "num_qubits": NUM_QUBITS,
            "measurement_model": variant.measurement_model,
            "pauli_settings": pauli_settings,
            "shots": SHOTS
        }

        with open(variant.metadata_path, "w") as f:
            json.dump(metadata, f, indent=2)


In [20]:
states = ["|0>", "|1>", "|+>", "|->", "|0+i1>/√2"]

In [21]:
base_dir = pathlib.Path("data")

variants = [
    DatasetVariant(
        name=s,
        circuit_summary=f"State preparation for {s}",
        measurement_model="Pauli projective",
        measurement_data_path=base_dir / f"single_qubit_{STATE_ID_MAP[s]}_measurements.npy",
        metadata_path=base_dir / f"single_qubit_{STATE_ID_MAP[s]}_metadata.json",
        density_matrix_path=base_dir / f"single_qubit_{STATE_ID_MAP[s]}_rho_true.npy"
    )
    for s in states
]
generate_measurement_dataset(variants)

Processing state: |0>
Saving to: data/single_qubit_zero_measurements.npy
Processing state: |1>
Saving to: data/single_qubit_one_measurements.npy
Processing state: |+>
Saving to: data/single_qubit_plus_measurements.npy
Processing state: |->
Saving to: data/single_qubit_minus_measurements.npy
Processing state: |0+i1>/√2
Saving to: data/single_qubit_phase_measurements.npy


In [22]:
print("Number of variants:", len(variants))
print(variants)

Number of variants: 5
[DatasetVariant(name='|0>', circuit_summary='State preparation for |0>', measurement_model='Pauli projective', measurement_data_path=PosixPath('data/single_qubit_zero_measurements.npy'), metadata_path=PosixPath('data/single_qubit_zero_metadata.json'), density_matrix_path=PosixPath('data/single_qubit_zero_rho_true.npy')), DatasetVariant(name='|1>', circuit_summary='State preparation for |1>', measurement_model='Pauli projective', measurement_data_path=PosixPath('data/single_qubit_one_measurements.npy'), metadata_path=PosixPath('data/single_qubit_one_metadata.json'), density_matrix_path=PosixPath('data/single_qubit_one_rho_true.npy')), DatasetVariant(name='|+>', circuit_summary='State preparation for |+>', measurement_model='Pauli projective', measurement_data_path=PosixPath('data/single_qubit_plus_measurements.npy'), metadata_path=PosixPath('data/single_qubit_plus_metadata.json'), density_matrix_path=PosixPath('data/single_qubit_plus_rho_true.npy')), DatasetVariant

In [23]:
!ls data
!ls data/single_qubit

single_qubit_minus_measurements.npy  single_qubit_phase_rho_true.npy
single_qubit_minus_metadata.json     single_qubit_plus_measurements.npy
single_qubit_minus_rho_true.npy      single_qubit_plus_metadata.json
single_qubit_one_measurements.npy    single_qubit_plus_rho_true.npy
single_qubit_one_metadata.json	     single_qubit_zero_measurements.npy
single_qubit_one_rho_true.npy	     single_qubit_zero_metadata.json
single_qubit_phase_measurements.npy  single_qubit_zero_rho_true.npy
single_qubit_phase_metadata.json
ls: cannot access 'data/single_qubit': No such file or directory


# Task 4: Single Qubit Tomography

In [24]:
def state_to_id(state_label: str) -> str:
    return {
        "|0>": "zero",
        "|1>": "one",
        "|+>": "plus",
        "|->": "minus",
        "|0+i1>/√2": "phase"
    }[state_label]

In [25]:
import pathlib
import numpy as np
import json

In [26]:
DATA_DIR = pathlib.Path("data")

states = ["|0>", "|1>", "|+>", "|->", "|0+i1>/√2"]


In [27]:
def load_measurement_data(state_label):
    path = DATA_DIR / f"single_qubit_{state_to_id(state_label)}_measurements.npy"
    print("Loading:", path)
    return np.load(path, allow_pickle=True).item()

In [28]:
def load_true_density_matrix(state_label):
    path = DATA_DIR / f"single_qubit_{state_to_id(state_label)}_rho_true.npy"
    print("Loading:", path)
    return np.load(path)

In [29]:
def linear_inversion_tomography(measurement_data):
    expX = measurement_data["X"]["probabilities"].get("0", 0) \
         - measurement_data["X"]["probabilities"].get("1", 0)

    expY = measurement_data["Y"]["probabilities"].get("0", 0) \
         - measurement_data["Y"]["probabilities"].get("1", 0)

    expZ = measurement_data["Z"]["probabilities"].get("0", 0) \
         - measurement_data["Z"]["probabilities"].get("1", 0)

    rho = 0.5 * (I + expX * X + expY * Y + expZ * Z)
    return rho

In [30]:
RECON_DIR = DATA_DIR / "reconstructions"
RECON_DIR.mkdir(parents=True, exist_ok=True)

reconstruction_results = {}

In [31]:
X=np.array([[0,1],[1,0]],dtype=complex)
Y=np.array([[0,-1j],[1j,0]],dtype=complex)
Z=np.array([[1,0],[0,-1]],dtype=complex)
I=np.array([[1,0],[0,1]],dtype=complex)

for state in states:
    meas = load_measurement_data(state)
    rho_recon = linear_inversion_tomography(meas)
    rho_true = load_true_density_matrix(state)

    state_id = state_to_id(state)
    np.save(RECON_DIR / f"{state_id}_rho_reconstructed.npy", rho_recon)

    reconstruction_results[state_id] = {
        "rho_reconstructed": rho_recon,
        "rho_true": rho_true
    }

Loading: data/single_qubit_zero_measurements.npy
Loading: data/single_qubit_zero_rho_true.npy
Loading: data/single_qubit_one_measurements.npy
Loading: data/single_qubit_one_rho_true.npy
Loading: data/single_qubit_plus_measurements.npy
Loading: data/single_qubit_plus_rho_true.npy
Loading: data/single_qubit_minus_measurements.npy
Loading: data/single_qubit_minus_rho_true.npy
Loading: data/single_qubit_phase_measurements.npy
Loading: data/single_qubit_phase_rho_true.npy


In [32]:
from scipy.linalg import sqrtm

In [33]:
def fidelity(rho, sigma):
    sqrt_rho = sqrtm(rho)
    return np.real(np.trace(sqrtm(sqrt_rho @ sigma @ sqrt_rho))) ** 2


In [34]:
def trace_distance(rho, sigma):
    diff = rho - sigma
    eigvals = np.linalg.eigvals(diff)
    return 0.5 * np.sum(np.abs(eigvals))


In [35]:
metrics = {}

for state in states:
    state_id = state_to_id(state)

    rho_r = reconstruction_results[state_id]["rho_reconstructed"]
    rho_t = reconstruction_results[state_id]["rho_true"]

    metrics[state_id] = {
        "fidelity": fidelity(rho_r, rho_t),
        "trace_distance": trace_distance(rho_r, rho_t),
        "trace_error": float(abs(np.trace(rho_r) - 1))
    }

In [36]:
METRICS_PATH = RECON_DIR / "tomography_metrics.json"

with open(METRICS_PATH, "w") as f:
    json.dump(metrics, f, indent=2)

print("Saved metrics to:", METRICS_PATH)


Saved metrics to: data/reconstructions/tomography_metrics.json


In [37]:
for state in states:
    state_id = state_to_id(state)
    rho_r = reconstruction_results[state_id]["rho_reconstructed"]

    plot_density_matrix_histogram(
        rho_r,
        title=f"Reconstructed density matrix ({state})"
    )


In [38]:
for k, v in metrics.items():
    print(f"{k:>6} | Fidelity: {v['fidelity']:.4f} | Trace dist: {v['trace_distance']:.4f}")

  zero | Fidelity: 1.0000 | Trace dist: 0.0100
   one | Fidelity: 1.0000 | Trace dist: 0.0085
  plus | Fidelity: 1.0000 | Trace dist: 0.0081
 minus | Fidelity: 1.0000 | Trace dist: 0.0202
 phase | Fidelity: 0.0000 | Trace dist: 1.0000


# Task 5: Validation & Reporting

In [39]:
from pathlib import Path
from typing import Sequence
import numpy as np
import json
import matplotlib.pyplot as plt

In [40]:
def summarize_validation_runs(result_paths: Sequence[Path]) -> None:
    """
    Aggregate tomography validation metrics (fidelity, trace distance, etc.)
    from stored result artifacts and generate summary tables and plots.
    """

    all_metrics = {}

    for path in result_paths:
        with open(path, "r") as f:
            metrics = json.load(f)
            all_metrics.update(metrics)

    # ---- Print summary table ----
    print("\nTomography Validation Summary")
    print("-" * 40)
    for state_id, vals in all_metrics.items():
        print(
            f"{state_id:>6} | "
            f"Fidelity: {vals['fidelity']:.4f} | "
            f"Trace dist: {vals['trace_distance']:.4f}"
        )

    # ---- Plot fidelities ----
    states = list(all_metrics.keys())
    fidelities = [all_metrics[s]["fidelity"] for s in states]
    trace_dists = [all_metrics[s]["trace_distance"] for s in states]

    plt.figure(figsize=(8, 4))
    plt.bar(states, fidelities)
    plt.ylim(0, 1.05)
    plt.ylabel("Fidelity")
    plt.title("Reconstruction Fidelity per State")
    plt.show()

    plt.figure(figsize=(8, 4))
    plt.bar(states, trace_dists)
    plt.ylabel("Trace Distance")
    plt.title("Trace Distance per State")
    plt.show()


In [41]:
RECON_DIR = Path("data/reconstructions")

for state in states:
    rho_path = RECON_DIR / f"{state_id}_rho_reconstructed.npy"
    print("Loading:", rho_path)

    rho = np.load(rho_path)

    plot_density_matrix_histogram(
        rho,
        title=f"Density matrix histogram — {state}"
    )


Loading: data/reconstructions/phase_rho_reconstructed.npy


Loading: data/reconstructions/phase_rho_reconstructed.npy


Loading: data/reconstructions/phase_rho_reconstructed.npy


Loading: data/reconstructions/phase_rho_reconstructed.npy


Loading: data/reconstructions/phase_rho_reconstructed.npy


In [42]:
import json
from pathlib import Path

METRICS_PATH = Path("data/reconstructions/tomography_metrics.json")

with open(METRICS_PATH, "r") as f:
    all_metrics = json.load(f)

all_metrics


{'zero': {'fidelity': 1.000000000252054,
  'trace_distance': 0.009962429422585618,
  'trace_error': 0.0},
 'one': {'fidelity': 1.000000000049715,
  'trace_distance': 0.008544003745317537,
  'trace_error': 0.0},
 'plus': {'fidelity': 1.0000000123996466,
  'trace_distance': 0.008062257748298558,
  'trace_error': 0.0},
 'minus': {'fidelity': 1.0000000002794742,
  'trace_distance': 0.020155644370746365,
  'trace_error': 0.0},
 'phase': {'fidelity': 5.545236260421639e-12,
  'trace_distance': 1.000025999662009,
  'trace_error': 0.0}}

In [43]:
import pandas as pd

# Convert metrics dict to DataFrame
metrics_df = pd.DataFrame.from_dict(all_metrics, orient="index")

# Rename columns for clarity
metrics_df = metrics_df.rename(columns={
    "fidelity": "Fidelity",
    "trace_distance": "Trace Distance",
    "trace_error": "Trace Error"
})

metrics_df


,Fidelity,Trace Distance,Trace Error
zero,1.000000e+00,0.009962,0.0
one,1.000000e+00,0.008544,0.0
plus,1.000000e+00,0.008062,0.0
minus,1.000000e+00,0.020156,0.0
phase,5.545236e-12,1.000026,0.0
